In [5]:
from __future__ import print_function
import sys, os
sys.path.append('/home/danilofrp/projeto_final/neural-stocks/src')
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralstocks.dataacquisition import *
from neuralstocks.preprocessing import *
from neuralstocks.plots import *
from neuralstocks.utils import *
%matplotlib inline

In [3]:
dataPath = '/home/danilofrp/projeto_final/data'
assetType = 'stocks'
asset = 'PETR4'
frequency = 'diario'

filePath = dataPath + '/' + assetType + '/' + asset + '/' + frequency + '/' + asset + '.CSV'

decomposeModel = 'additive'

saveDir = '/home/danilofrp/projeto_final/results/preprocessing/misc'
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
saveFormat = 'png'

plt.rcParams['font.weight'] = 'bold'
plt.rcParams['figure.titlesize'] = 18
plt.rcParams['figure.titleweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13

In [7]:
init_time = time.time()
df = acquireData(filePath = filePath, replicateForHolidays = True)
end_time = time.time()
print ('Time to acquire data: '+str(end_time-init_time)+' seconds')

Time to acquire data: 2.31478190422 seconds


In [ ]:
init_time = time.time()
df2 = df.copy()
windowMaxSize = 15
weightModelWindowMaxSize = 100
model = 'additive'
column = 'Close'
minimal = None
RMSE = np.empty((windowMaxSize + 1, weightModelWindowMaxSize + 1), dtype=float)*np.nan
for i in range(2, windowMaxSize + 1):
    for j in range(2 * i, weightModelWindowMaxSize + 1):
        i_time = time.time()
        deTrend(df2, column = column, window = i, model = model, fitOrder = 1, weightModel = 'window_acorr', weightModelWindow = j)
        e_time = time.time()
        print('Completed deTrend ({}, {}). Time to run deTrend: {} seconds'.format(i, j, str(e_time-i_time)), end='\r')
        if model.startswith('m'):
            RMSE[i, j] = np.square((df2['{}_resid'.format(column)] - 1)).sum()/len(df2['{}_resid'.format(column)].dropna())
        else:
            RMSE[i, j] = np.square(df2['{}_resid'.format(column)]).sum()/len(df2['{}_resid'.format(column)].dropna())
        if not minimal:
            minimal = RMSE[i][j]
            iMin = i
            jMin = j
        if RMSE[i, j] < minimal:
            minimal = RMSE[i][j]
            iMin = i
            jMin = j

print('Minimal RSME = {}, at window = {} and weightModelWindow = {} * window'.format(minimal, iMin, jMin))
RMSE_backup = RMSE

end_time = time.time()
print('Time to run {}x{} acorr deTrend analysis: {} seconds'.format(windowMaxSize, weightModelWindowMaxSize, str(end_time-init_time)))


/home/danilofrp/projeto_final/neural-stocks/src/neuralstocks/utils.py:41: RankWarning: Polyfit may be poorly conditioned
  a = np.polyfit(x, y, deg = fitOrder, w = weights)
/home/danilofrp/projeto_final/neural-stocks/src/neuralstocks/utils.py:41: RankWarning: Polyfit may be poorly conditioned
  a = np.polyfit(x, y, deg = fitOrder, w = weights)


/home/danilofrp/projeto_final/neural-stocks/src/neuralstocks/utils.py:41: RankWarning: Polyfit may be poorly conditioned
  a = np.polyfit(x, y, deg = fitOrder, w = weights)


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
cax = ax.imshow(RMSE[:,:], cmap="jet", aspect="auto")
plt.gca().invert_yaxis()
ax.set_title('RMSE analysis: ACF sweep x Trend sweep')
ax.set_ylim([3.5,10.5])
ax.set_xlim([5.5,50.5])
ax.set_xlabel('Delayed samples used for ACF estimation')
ax.set_ylabel('Delayed samples used for trend estimation')
cbar = fig.colorbar(cax)
plt.figtext(0.5,  0.010, 'Minimal RSME: {:.3f}, using Trend window with {} samples \nand ACF window with {} samples'.format(minimal, iMin, jMin), size = 14, horizontalalignment = 'center')
fig.savefig('{}/{}.{}'.format(saveDir, 'PETR4_RMSEanalysis_acf', 'pdf'), bbox_inches='tight')
